In [5]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import helpers.algorithms_utils as alg_utils
from IPython.display import HTML
import matplotlib.pyplot as plt
import multiprocessing
from helpers.mv_single_binary import majority_voting
from algorithms.smart_stop import decision_function_bayes

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### Vote Adquisition Function

In [6]:
def generate_votes_smart_stop(params, items_num, ct, gt, decision_fn):
    total_votes = {}
    
    #base votes
    for i in range(items_num):
        total_votes[i] = {}
        for k in range(params['votes_per_item']):
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)

            total_votes[i][worker_id] = [vote]
            
    #evaluate votes
    results = decision_fn(items_num, total_votes, ct, params['cost_ratio'], 
                                                   params['classification_fn'])
    
    #Check if must continue collecting votes
    items_predicted_classified = alg_utils.get_items_predicted_classified(results)
    must_get_more_votes = len(items_predicted_classified) > 0
     
    while(must_get_more_votes):
        total_votes_aux = {}
        for i, v in items_predicted_classified.items():           
            worker_id, vote = alg_utils.get_worker_vote(params['workers_accuracy'], i, gt, total_votes)
            
            total_votes[i][worker_id] = [vote]
            
            total_votes_aux[i] = total_votes[i].copy()
        #end for
        
        results = decision_fn(len(total_votes_aux), total_votes_aux, ct, params['cost_ratio'], 
                                                   params['classification_fn'])

        #Stop when there are no more items that can be classified
        items_predicted_classified = alg_utils.get_items_predicted_classified(results)
        must_get_more_votes = len(items_predicted_classified) > 0
    #end while
    
    items_classification = alg_utils.classify_items(total_votes, gt, majority_voting, ct)

    return items_classification, total_votes

### Experiment Runner Function

In [7]:
def run_experiment_no_multiprocessing(generate_votes_fn, df, cf, data_true_percentage):
    
    main_results = []
    
    for ct in cts:
        ct = round(ct, 2) #limit to two decimals
        losses = []
        recalls = []
        precisions = []
        costs = []
        f_ones = []
        f_betas = []
        wces = []

        for _ in range(iterations_per_ct):
            workers_accuracy = alg_utils.simulate_workers(workers_num, z, fixed_acc, fixed_workers_acc, base_workers_acc, max_workers_acc)
    
            params = {
                'workers_accuracy': workers_accuracy,
                'workers_num': workers_num,
                'items_num': items_num,
                'cost_ratio': cr,
                'votes_per_item': base_votes_per_item,
                'classification_fn': cf
            }
            
            items_classification, total_votes = generate_votes_smart_stop(params, items_num, ct, items_ground_truth, df)

            costs.append(np.mean([len(v) for k,v in total_votes.items()]))

            loss, recall, precision, f1, beta, f_beta, wce = alg_utils.Metrics.compute_metrics_full(items_classification, items_ground_truth, fnc, fpc)
            losses.append(loss)
            recalls.append(recall)
            precisions.append(precision)
            f_ones.append(f1)
            f_betas.append(f_beta)
            wces.append(wce)
        #end for iterations
        
        results = [['adaptive_em',
                             dataset_name,
                             cr,
                             str(cf.__name__),
                             str(df.__name__),
                             ct,
                             workers_num,
                             f'U({base_workers_acc},{max_workers_acc})',
                             items_num,
                             data_true_percentage,
                             round(np.mean(costs), 3), 
                             round(np.std(costs), 3),
                             round(np.mean(recalls), 3),
                             round(np.std(recalls), 3),
                             round(np.mean(precisions), 3),
                             round(np.std(precisions), 3),           
                             round(np.mean(losses), 3),
                             round(np.std(losses), 3),                       
                             round(np.mean(f_ones), 3),
                             round(np.std(f_ones), 3),                        
                             round(np.mean(f_betas), 3),
                             round(np.std(f_betas), 3),    
                             round(np.mean(wces), 3),
                             round(np.std(wces), 3),
                             fnc,
                             fpc,
                             ucc
                             ]]
        
    pd.DataFrame(results, columns=pdColumns).to_csv(f'results/adaptive_em/results.csv', mode='a', index=False, header=False)
        
    
    return []


### Real-World Dataset Processing Function

In [8]:
def get_dataset_info(dataset, path):
    dataset_name = dataset['name']
    dataset_gt = dataset['gt_file']
    dataset_votes = dataset['votes_file']
    dataset_delimiter = dataset['delimiter']
    dataset_header = dataset['header']

    gt_index_item = dataset['gt_index_item']
    gt_index_value = dataset['gt_index_value']
    votes_index_worker = dataset['votes_index_worker']
    votes_index_item = dataset['votes_index_item']
    votes_index_vote = dataset['votes_index_vote']

    gt = pd.read_csv(f'{path}/{dataset_name}/{dataset_gt}', delimiter=dataset_delimiter, header=dataset_header)
    votes = pd.read_csv(f'{path}/{dataset_name}/{dataset_votes}', delimiter=dataset_delimiter, header=dataset_header)
    
    items_num = len(gt[gt_index_item].unique())
    votes_num = len(votes)
    gt_classes = gt[gt_index_value].unique()
    gt_balance = {}

    for gt_class in gt_classes:
        class_items = gt[gt[gt_index_value] == gt_class]
        gt_balance[gt_class] = len(class_items) / items_num
        
    workers_id = votes[votes_index_worker].unique()

    workers_acc = {}
    workers_acc_indexed = []
    workers_answers_correctness = {}
    for worker_id in workers_id:

        worker_votes = votes[votes[votes_index_worker] == worker_id]
        workers_answers_correctness[worker_id] = []
        for key, worker_vote in worker_votes.iterrows():

            item_id = worker_vote[votes_index_item]
            vote = worker_vote[votes_index_vote]
            item_gt = gt[gt[gt_index_item] == item_id][gt_index_value]

            if item_gt.empty == False:
                workers_answers_correctness[worker_id].append(int(vote == item_gt.iloc[0]))
        #end for votes

        if len(workers_answers_correctness[worker_id]) != 0:
            worker_acc = np.mean(workers_answers_correctness[worker_id])
            workers_acc[worker_id] = worker_acc
            workers_acc_indexed.append(worker_acc)
    #end for workers

    avg_acc = np.mean(workers_acc_indexed)
    workers_num = len(workers_acc_indexed)
    
    return items_num, gt_balance, votes_num, workers_num, avg_acc


## Experiments

### Datasets

In [14]:
datasets = [
            {'name':'HITspam', 'gt_file':'gold.txt','votes_file':'labels.txt', 'gt_index_item': 0, 'gt_index_value': 1, 'votes_index_worker': 0, 
             'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':'\t', 'header':None},
            {'name':'TEMP', 'gt_file':'gold_and_votes.txt','votes_file':'gold_and_votes.txt', 'gt_index_item': 2, 'gt_index_value': 4, 
             'votes_index_worker': 1, 'votes_index_item': 2, 'votes_index_vote': 3, 'delimiter':'\t', 'header':None},
            {'name':'WVSCM', 'gt_file':'groundTruth.txt','votes_file':'labels.txt', 'gt_index_item': 0, 'gt_index_value': 1, 'votes_index_worker': 0, 
             'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':'\t', 'header':None},  
            {'name':'WaterBird1', 'gt_file':'gt.txt','votes_file':'labels.txt', 'gt_index_item': 0, 'gt_index_value': 1, 'votes_index_worker': 0, 
             'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':'\t', 'header':None}, 
            {'name':'TlkAgg2', 'gt_file':'golden_labels.tsv','votes_file':'crowd_labels.tsv', 'gt_index_item': 0, 'gt_index_value': 1, 
             'votes_index_worker': 0, 'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':'\t', 'header':None},
            {'name':'Div150Multi', 'gt_file':'mediaEval1Q1_Gold.txt','votes_file':'mediaEvalQ1_Responses.txt', 'gt_index_item': 0, 'gt_index_value': 1, 'votes_index_worker': 0, 
             'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':' ', 'header':None},
            {'name':'HC-TREC-2011', 'gt_file':'trec2011Task2_Gold.txt','votes_file':'trec2011Task2_Responses.txt', 'gt_index_item': 0, 
             'gt_index_value': 1, 'votes_index_worker': 0, 'votes_index_item': 1, 'votes_index_vote': 2, 'delimiter':' ', 'header':None}
           ]

### Experiment settings

In [10]:
pdColumns = ["fn_name","dataset","cost_ratio", "class_fn", "decision_fn","threshold",'num_workers',
             'workers_distribution','num_items','data_bal','cost','cost_std', 'recall','recall_std', 'precision', 
             'precision_std', 'loss', 'loss_std', 'f1', 'f1_std', 'fbeta', 'fbeta_std', 'wce','wce_std','fnc','fpc', 'ucc']


fnc = 1
fpc = 1
ucc = 1

#cost ratios
crs = [1/10, 1/20, 1/50, 1/50]

#crowd
base_votes_per_item = 3
z = 0 #% cheaters

fixed_acc = False
fixed_workers_acc = None

#ground truth 
#items_num = 1000

cts = [.7, .8, .9, .95]
iterations_per_ct = 10

decision_function = decision_function_bayes
cf = majority_voting

datasets_path = '/Users/pmaglione/Repos/crowd-datasets'

In [12]:
def get_dataset_true_positive_percentage(balance):
    if 'Yes' in balance.keys():
        return balance['Yes']
    elif 1 in balance.keys():
        return balance[1]

In [15]:
datasets = [datasets[2]]
for dataset in datasets:
    items_num, gt_balance, votes_num, workers_num, avg_acc = get_dataset_info(dataset, datasets_path)
    
    #we only use binary class datasets
    if len(gt_balance) == 2:
        dataset_name = dataset['name']
        base_workers_acc = avg_acc - (1 - avg_acc) / 2
        max_workers_acc = avg_acc + (1 - avg_acc) / 2
        
        workers_num = 2000

        data_true_percentage = get_dataset_true_positive_percentage(gt_balance)
        items_ground_truth = alg_utils.generate_gold_data(items_num, data_true_percentage)
        
        for cr in crs:
            _ = run_experiment_no_multiprocessing(generate_votes_smart_stop, decision_function, cf, data_true_percentage)


ValueError: Shape of passed values is (1, 27), indices imply (27, 27)

In [18]:
r = [1] * 27
results = [r]
pd.DataFrame(results, columns=pdColumns).to_csv(f'results/adaptive_em/results.csv', mode='a', index=False, header=False)